# 实现神经网络的前向传播

## 导入MNIST数据集

In [16]:
from datasets.mnist import load_mnist
import numpy as np
# 第一次调用会花费几分钟 ……
(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=True)
# 输出各个数据的形状
print(x_train.shape) # (60000, 784)
print(y_train.shape) # (60000,)
print(x_test.shape) # (10000, 784)
print(y_test.shape) # (10000,)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


## 实现神经网络的前向传播

1. 实现激活函数（relu、sigmoid、softmax）
2. 实现三层全连接神经网络
3. 使用神经网络进行前向传播
4. 加载已训练好的神经网络进行推理

In [11]:
# 激活函数
def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    x_max = np.max(x, axis=-1, keepdims=True)
    x = x-x_max  #防止溢出
    exp_x = np.exp(x)
    return exp_x/np.sum(exp_x, axis=-1, keepdims=True)

### 三层全连接网络

这里实现的三层全连接网络模型细节如下
- 神经网络的输入层有784个神经元，输出层有10个神经元。
- 输入层的784这个数字来源于图像大小的28 × 28 = 784。
- 输出层的10这个数字来源于10类别分类（数字0到9，共10类别）。-
- 神经网络有2个隐藏层，第1个隐藏层有50个神经元，第2个隐藏层有100个神经元。

In [12]:
# 初始化神经网络参数
def init_network():
    network = {}
    network['W1'] = np.random.normal(size=(784, 50))
    network['b1'] = np.random.normal(size=(50,))
    network['W2'] = np.random.normal(size=(50, 100))
    network['b2'] = np.random.normal(size=(100,))
    network['W3'] = np.random.normal(size=(100, 10))
    network['b3'] = np.random.normal(size=(10,))
    return network

# 前向传播
def forward(netwok, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1)+b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2)+b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3)+b3
    y = softmax(a3)
    return y

In [13]:
# 使用神经网络进行前向传播
network = init_network()
y = forward(network, x_test)
print(y.shape)
# 计算准确性
accuracy_cnt = (np.argmax(y, axis=1)==y_test).sum()
print(f"accuracy:{accuracy_cnt/len(y)}")

(10000, 10)
accuracy:0.1137


### 加载已训练好的模型进行推断

从上面可以看到随机初始化的神经网络（尚未训练）准确率约为10%（随机猜测的结果）

由于这一节尚未涉及训练过程，这里将使用一个已训练好的权重，看一下效果

已训练好的权重位于"weights/sample_weight.pkl"

In [15]:
import pickle
with open("weights/sample_weight.pkl", 'rb') as f:
    network = pickle.load(f)
y = forward(network, x_test)
# 计算准确性
accuracy_cnt = (np.argmax(y, axis=1)==y_test).sum()
print(f"accuracy:{accuracy_cnt/len(y)}")

accuracy:0.9352


### 实现批处理

In [19]:
batch_size = 1024
accuracy_cnt = 0
for i in range(0, len(x_test), batch_size):
    x_batch = x_test[i:i+batch_size]
    y_batch = y_test[i:i+batch_size]
    y = forward(network, x_batch)
    accuracy_cnt += (np.argmax(y, axis=1)==y_batch).sum()
print(accuracy_cnt/len(x_test))

0.9352
